In [ ]:
import pandas as pd
import numpy as np
state_df = pd.read_csv('./statewide.csv')
wf_df = pd.read_csv('./Wildfires_With_Weather.csv')
print(wf_df.head())
(state_df.head())

   Started  Year  Month     County  AcresBurned  Injuries  Fatalities  \
0   201301  2013      1       Lake          296         0           0   
1   201302  2013      2       Inyo          406         2           0   
2   201302  2013      2  Riverside          311         0           0   
3   201304  2013      4     Madera          274         0           0   
4   201304  2013      4     Fresno           80         0           0   

   StructuresDestroyed  StructuresDamaged  PropetyValue_Damage  Drought_Index  \
0                    0                  0                  0.0          -1.05   
1                    0                  0                  0.0          -3.33   
2                    0                  0                  0.0          -2.14   
3                    0                  0                  0.0          -3.65   
4                    0                  0                  0.0          -4.66   

   Precipitation  Temperature  Heating_Degree_Days  Cooling_Degree_Days  


,Year,Month,PropertyValueDamage,Temperature,Temperature_1,Temperature_2,Precipitation,Precipitation_1,Precipitation_2,PDSI,PDSI_1,PDSI_2,n,n_1,n_2
0,2013,1,0.0,42.1,43.0,52.5,1.19,6.15,3.57,-0.80,1.08,0.24,1,0,3
1,2013,2,0.0,45.3,42.1,43.0,0.58,1.19,6.15,-1.66,-0.80,1.08,2,1,0
2,2013,3,0.0,53.8,45.3,42.1,1.33,0.58,1.19,-2.43,-1.66,-0.80,0,2,1
3,2013,4,0.0,58.2,53.8,45.3,0.71,1.33,0.58,-2.83,-2.43,-1.66,4,0,2
4,2013,5,7053960.0,63.8,58.2,53.8,0.62,0.71,1.33,-2.98,-2.83,-2.43,15,4,0


In [ ]:
df = state_df.groupby('Month')['PropertyValueDamage'].sum()
df2 = state_df.groupby('Month')['n'].sum()
df = pd.merge(df, df2, on='Month', how='left')
df = df['PropertyValueDamage'] / df['n']
month_to_damge = df.to_dict()
print(month_to_damge)

{1: 0.0, 2: 8178884.166666667, 3: 0.0, 4: 3561.0714285714284, 5: 435730.26666666666, 6: 663542.9625, 7: 8576920.206185566, 8: 1995200.5867088607, 9: 1310732.7007746478, 10: 130877336.82926829, 11: 244846189.4117647, 12: 163789743.1818182}


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from statistics import mean
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import time

train, test = train_test_split(state_df, test_size=0.3, random_state=1)

x_train = train.drop(['Year', 'Month', 'n'], axis=1)
x_test = test.drop(['Year', 'Month', 'n'], axis=1)
y_train = train['n'].to_frame()
y_test = test['n'].to_frame()

y_scaler = MinMaxScaler(feature_range=(0, 1))
y_train_rescaled = y_scaler.fit_transform(y_train)
y_train_rescaled = pd.DataFrame(data = y_train_rescaled, columns = y_train.columns)

y_test_rescaled = y_scaler.transform(y_test)
y_test_rescaled = pd.DataFrame(data = y_test_rescaled, columns = y_test.columns)

weather_cols = [['Temperature', 'Precipitation', 'PDSI'], ['Temperature', 'Temperature_1', 'Precipitation', 'Precipitation_1', 'PDSI', 'PDSI_1'], ['Temperature', 'Temperature_1', 'Temperature_2', 'Precipitation', 'Precipitation_1', 'Precipitation_2', 'PDSI', 'PDSI_1', 'PDSI_2']]

freq_cols =[['n_1'], ['n_1', 'n_2']]

hybrid_cols = [['n_1', 'Temperature', 'Precipitation', 'PDSI'], ['n_1', 'n_2', 'Temperature', 'Temperature_1', 'Precipitation', 'Precipitation_1', 'PDSI', 'PDSI_1'], ['n_1', 'n_2', 'Temperature', 'Temperature_1', 'Temperature_2', 'Precipitation', 'Precipitation_1', 'Precipitation_2', 'PDSI', 'PDSI_1', 'PDSI_2']]

def lrTrain(cols):
  start_time = time.time()
  x_train_2 = x_train[cols]
  x_test_2 = x_test[cols]

  x_scaler = MinMaxScaler(feature_range=(0, 1))
  x_train_rescaled = x_scaler.fit_transform(x_train_2)
  x_train_rescaled = pd.DataFrame(data = x_train_rescaled, columns = x_train_2.columns)

  x_test_rescaled = x_scaler.transform(x_test_2)
  x_test_rescaled = pd.DataFrame(data = x_test_rescaled, columns = x_test_2.columns)

  lr = LinearRegression()

  lr.fit(x_train_rescaled, y_train_rescaled)
  pred = lr.predict(x_test_rescaled)
  mse = mean_squared_error(y_test_rescaled, pred)
  r2 = r2_score(y_test_rescaled, pred)

  end_time = time.time()
  elapsed_time = end_time - start_time

  result = pd.DataFrame({"Model": [cols], "R-Squared": [r2], "MSE": [mse], 'Prediction Time': [elapsed_time]})
  print(result)
  return result

results_df = pd.DataFrame({})
for i in range(len(weather_cols)):
  results_df = pd.concat([results_df, lrTrain(weather_cols[i])])
for i in range(len(freq_cols)):
  results_df = pd.concat([results_df, lrTrain(freq_cols[i])])
for i in range(len(hybrid_cols)):
  results_df = pd.concat([results_df, lrTrain(hybrid_cols[i])])
print(results_df)

                                Model  R-Squared       MSE  Prediction Time
0  [Temperature, Precipitation, PDSI]    0.77684  0.020908         0.043722
                                               Model  R-Squared       MSE  \
0  [Temperature, Temperature_1, Precipitation, Pr...   0.788694  0.019797   

   Prediction Time  
0         0.012819  
                                               Model  R-Squared      MSE  \
0  [Temperature, Temperature_1, Temperature_2, Pr...   0.795606  0.01915   

   Prediction Time  
0         0.012495  
   Model  R-Squared       MSE  Prediction Time
0  [n_1]   0.701137  0.028001         0.009868
        Model  R-Squared       MSE  Prediction Time
0  [n_1, n_2]   0.754073  0.023041         0.010678
                                     Model  R-Squared       MSE  \
0  [n_1, Temperature, Precipitation, PDSI]   0.839451  0.015042   

   Prediction Time  
0         0.011295  
                                               Model  R-Squared       MSE  \
0  [

In [ ]:
results_df.to_csv('./results.csv')

In [ ]:
freq = wf_df.groupby('County').size().to_frame('n')
freq['Percent'] = freq['n'] / freq['n'].sum()
freq_map = freq['Percent'].to_dict()
freq_map['Imperial'] = 0
freq_map['Sanfrancisco'] = 0
print(freq_map)


{'Alameda': 0.023127753303964757, 'Alpine': 0.0022026431718061676, 'Amador': 0.012114537444933921, 'Butte': 0.034140969162995596, 'Calaveras': 0.015418502202643172, 'Colusa': 0.004405286343612335, 'Contracosta': 0.016519823788546256, 'Delnorte': 0.006607929515418502, 'Eldorado': 0.022026431718061675, 'Fresno': 0.030837004405286344, 'Glenn': 0.007709251101321586, 'Humboldt': 0.016519823788546256, 'Inyo': 0.011013215859030838, 'Kern': 0.031938325991189426, 'Kings': 0.005506607929515419, 'Lake': 0.02973568281938326, 'Lassen': 0.018722466960352423, 'Losangeles': 0.02753303964757709, 'Madera': 0.023127753303964757, 'Marin': 0.005506607929515419, 'Mariposa': 0.022026431718061675, 'Mendocino': 0.02092511013215859, 'Merced': 0.011013215859030838, 'Modoc': 0.01762114537444934, 'Mono': 0.00881057268722467, 'Monterey': 0.028634361233480177, 'Napa': 0.022026431718061675, 'Nevada': 0.013215859030837005, 'Orange': 0.009911894273127754, 'Placer': 0.016519823788546256, 'Plumas': 0.009911894273127754, 

In [ ]:
damage = wf_df.groupby('County')['PropetyValue_Damage'].sum().to_frame('TotalDamage')
damage['Percent'] = damage['TotalDamage'] / damage['TotalDamage'].sum()
damage_map = damage['Percent'].to_dict()
damage_map['Imperial'] = 0
damage_map['Sanfrancisco'] = 0
print(damage_map)

{'Alameda': 4.411212107776137e-05, 'Alpine': 0.0, 'Amador': 0.0007277208891360058, 'Butte': 0.26773887479097636, 'Calaveras': 0.00012303662434956262, 'Colusa': 0.006348917719021216, 'Contracosta': 2.926462666622218e-05, 'Delnorte': 0.0, 'Eldorado': 0.0025973716110687848, 'Fresno': 0.00018359163539054482, 'Glenn': 0.002497723405459514, 'Humboldt': 0.00012025609749033034, 'Inyo': 0.001453171617894595, 'Kern': 0.0, 'Kings': 9.554039882207829e-06, 'Lake': 0.009583159907189347, 'Lassen': 8.502363879498311e-05, 'Losangeles': 0.047408932544009436, 'Madera': 0.0011424578871632358, 'Marin': 0.0, 'Mariposa': 0.0017849274759232816, 'Mendocino': 0.014801391865485972, 'Merced': 0.0, 'Modoc': 5.474206635210972e-05, 'Mono': 0.0017147223896847891, 'Monterey': 0.0004234763623465092, 'Napa': 0.21858599234935194, 'Nevada': 0.0010959387505421777, 'Orange': 0.0026419502447245784, 'Placer': 0.00016361078117202479, 'Plumas': 0.0, 'Riverside': 0.0023151043885434644, 'Sacramento': 0.0, 'Sanbenito': 4.402604864

In [ ]:
import pickle
with open("../server/models/firePredictor.pkl", "wb") as f:
    pickle.dump(lr, f)

with open("../server/models/x_scaler.pkl", "wb") as f:
    pickle.dump(x_scaler, f)

with open("../server/models/y_scaler.pkl", "wb") as f:
    pickle.dump(y_scaler, f)

with open("../server/models/all_damages.pkl", "wb") as f:
    pickle.dump(month_to_damge, f)

with open("../server/models/countyPredictor.pkl", "wb") as f:
    pickle.dump(freq_map, f)

with open("../server/models/county_damages.pkl", "wb") as f:
    pickle.dump(damage_map, f)